# Transmission line circuit

We consider the transmission line model used in  [AKS11]. The RLC circuit of the transmission line is shown below.

<img src="tline.png" alt="Transmission line circuit" style="width: 700px;"/>

Here, $U_{in}$ is the voltage at the sending end, and $U_{out}$ is the voltage at the receiving end.

## Model description

The pre-requisite to understand this model is any standard textbook on electrical circuits. The electrical elements law for resistors (R), inductors (L) and capaicitors (C) are studied, for instance, in [Chapter3, K15].

Let's assume that the network consists of $\eta > 2$ nodes. There are $\eta$ dynamic equations for the capacitor voltages and $\eta$ dynamic equations for the circuit currents. Therefore, the state vector can be represented as

$$
x = [U_1, U_2, \ldots, U_\eta, I_1, I_2, \ldots, I_\eta]^T,
$$
and the state dimension is $2\eta$.

### First node

When writing the equations for the voltages and currents, we should pay attention to the sign choices in the circuit, which are slighyly non-stadard ($U_1$'s positive terminal is at the bottom, etc. as indicated by the arrows).

Let $R_d$ denote the driver resistance's current.

By Kirchhoff's voltage law,

$$
U_{in} = R_d I_1 + LI_1' - U_1,
$$
therefore
$$
\boxed{I_1' = \dfrac{U_{in} + U_1}{L} - \dfrac{R_d}{L}I_1}.
$$

By Kirchhoff's current law, and if $I_{1, C}$ denotes the current through the capacitor $C$, connected to the first node,

$I_1 = I_2 + I_{C, 1}$, and $I_{C, 1} = -CU_1'$, then

$$
\boxed{U_1' = \frac{I_2 - I_1}{C}}.
$$

### Other nodes

Other node's equations are obtained in a similar fashion. For instance, for the second loop one has

$$
-U_1 = RI_2 + LI_2' - U_2 \Rightarrow I_2' = \dfrac{U_2-U_1}{L} - \frac{R}{L}I_2
$$
for the current's equation, and 
$$
I_{2, C} = I_2 - I_3,\Rightarrow U_2' = \frac{I_3 - I_2}{C}.
$$
for the voltage's equation

Generalizing for arbitrary $l = 2, \ldots, \eta - 1$ is trivial and gives:

$$
\boxed{I'_l = \dfrac{U_l - U_{l-1}}{L} - \frac{R}{L}I_l}.
$$

$$
\boxed{U'_l = \dfrac{I_{l+1} - I_l}{C}}.
$$

### Last node

The last node corresponds to the case $l = \eta$

$$
\boxed{I'_\eta = \dfrac{U_\eta - U_{\eta-1}}{L} - \frac{R}{L}I_\eta}.
$$

$$
\boxed{U'_{out} = U'_{out} = - \frac{I_{\eta}}{C}}
$$

### System of differential equations

Recall that that the stateset is $\mathbb{R}^{2\eta}$, where the state variables are $x = [U_1, U_2, \ldots, U_\eta, I_1, I_2, \ldots, I_\eta]^T$. The system can be written as a block-diagonal system of linear ODEs, 

$$
x'(t) = Ax(t) + BU_{in}(t),
$$
using the results in the previous section. There exist useful constructors in `Base.LinearAlgebra` to build matrix with special shape, such as diagonal and band matrices.

In [1]:
using MathematicalSystems, LinearAlgebra, SparseArrays

function tline(;η=3, R=1.00, Rd=10.0, L=1e-10, C=1e-13 * 4.00)
    A₁₁ = zeros(η, η)
    A₁₂ = Bidiagonal(fill(-1/C, η), fill(1/C, η-1), :U)
    A₂₁ = Diagonal(vcat(-Rd/L, fill(-R/L, η-1)))
    A₂₂ = Bidiagonal(fill(1/L, η), fill(-1/L, η-1), :L)
    A  = [A₁₁ A₁₂; A₂₁ A₂₂]
    B = sparse([η+1], [1], one(eltype(A)), 2η, 1)
    return A, B
end

tline (generic function with 1 method)

For example, we can visualize the matrices for the case $\eta = 3$ using `ModelingToolkit`:

In [2]:
using ModelingToolkit

@variables R Rd L C
A, B = tline(R=R, Rd=Rd, L=L, C=C)
Matrix(A)

6×6 Array{Expression,2}:
 Constant(0)  Constant(0)  Constant(0)       -1 / C        1 / C  Constant(0)
 Constant(0)  Constant(0)  Constant(0)  Constant(0)       -1 / C        1 / C
 Constant(0)  Constant(0)  Constant(0)  Constant(0)  Constant(0)       -1 / C
     -Rd / L  Constant(0)  Constant(0)        1 / L  Constant(0)  Constant(0)
 Constant(0)       -R / L  Constant(0)       -1 / L        1 / L  Constant(0)
 Constant(0)  Constant(0)       -R / L  Constant(0)       -1 / L        1 / L

If we now use the default values,

In [4]:
A, B = tline(η=3)
Matrix(A)

6×6 Array{Float64,2}:
  0.0      0.0      0.0     -2.5e12   2.5e12   0.0   
  0.0      0.0      0.0      0.0     -2.5e12   2.5e12
  0.0      0.0      0.0      0.0      0.0     -2.5e12
 -1.0e11   0.0      0.0      1.0e10   0.0      0.0   
  0.0     -1.0e10   0.0     -1.0e10   1.0e10   0.0   
  0.0      0.0     -1.0e10   0.0     -1.0e10   1.0e10

Notice that the matrix coefficients are big, and taking the matrix exponential would easily overflow, unless the time-step is of the order of `1e-10`.

Hence, it is convenient to rescale the system. Let $\alpha > 0$ be a scaling factor, and let $\tilde{x}(t) = x(\alpha t)$. By the chain rule, 

$$
\tilde{x}'(t) = \alpha x'(\alpha t) = \alpha A x(\alpha t) + \alpha B U_{in}(\alpha t) = \tilde{A} \tilde{x}(t) + \tilde{B} \tilde{U}_{in}(t),
$$
where $\tilde{A} := \alpha A$ and $\tilde{B} := \alpha B$.

In [13]:
function scale!(s::LinearControlContinuousSystem, α=1.0)
    s.A .*= α
    s.B .*= α
    return s
end

scale! (generic function with 2 methods)

In [21]:
A, B = tline(η=3)
s = LinearControlContinuousSystem(A, B)
scale!(s, 1e-9);

In [19]:
Matrix(s.A)

6×6 Array{Float64,2}:
   0.0   0.0   0.0  -250.0   250.0     0.0
   0.0   0.0   0.0     0.0  -250.0   250.0
   0.0   0.0   0.0     0.0     0.0  -250.0
 -10.0   0.0   0.0     1.0     0.0     0.0
   0.0  -1.0   0.0    -1.0     1.0     0.0
   0.0   0.0  -1.0     0.0    -1.0     1.0

Note that under this transformation, the time horizon has to be transformed as well, through $\tilde{T} = \alpha T$.

## Reachability settings

The transmission line parameters used in this model are displayed in the following table.

|resistance in [Ω] | driver resistance in [Ω] | Inductance in [H]| Capacitance in [F]|
|------|-----|------|-----|
|R = 1.00| Rdriver = 10.0| L = 1e−10 | C = 4e−13|

The steady state is obtained by zeroing the left-hand side of the ODE, obtaining

$$
0 = x' = Ax_\infty + Bu_0 \Rightarrow x_\infty = -A^{-1}B u_0 
$$

The initial set under consideration corresponds to the steady state for input voltages $U_{in, ss} := [-0.2, 0.2]$. Moreover, an uncertainty is added so taht the initial currents are also uncertain. The set of initial states is then

$$
x(0) \in \mathcal{X}_0 := -A^{-1} B  U_{in, ss} \oplus □(0.001),
$$
where $□(ϵ)$ is the infinity-norm ball of center zero and radius $ϵ$.

The time horizon is $T = 0.7s$. For modeling, we consider the scaling parameter $\alpha = 1e-9$.

In [27]:
using LazySets

η = 20 # order
A, B = tline(η=η)
s = LinearControlContinuousSystem(A, B)
scale!(s, 1e-9);

Uin_ss = Interval(-0.2, 0.2)
□(ϵ) = BallInf(zeros(2η), ϵ)
X0 = -inv(Matrix(A))*B*Uin_ss ⊕ □(0.001)

MinkowskiSum{Float64,LazySets.LinearMap{Float64,Interval{Float64,IntervalArithmetic.Interval{Float64}},Float64,Array{Float64,2}},BallInf{Float64}}(LazySets.LinearMap{Float64,Interval{Float64,IntervalArithmetic.Interval{Float64}},Float64,Array{Float64,2}}([1.0000000000000001e-11; 0.0; … ; 0.0; 0.0], Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.200001, 0.200001])), BallInf{Float64}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.001))

Initially, the system is supposed to be in an -neighborhood
(with  = 0.01) of its steady state for an input voltage
inside [−0.2; 0.2]. Then, at time t = 0, the input voltage is
switched to a value in [0.99; 1.01]:
I = −A
−1
b[−0.2; 0.2] ⊕ B U = [0.9; 1.1]

We consider a scaling factor $\alpha = 1e-9$. $T' = \alpha 

The initial state of the system is determined by the steady state
solution for input voltages Uin = u ∈ [−0.2,0.2] to which an uncertainty is added
so that the initial currents are also uncertain: R(0) = −A
−1bu + (0.001), where
A, b are chosen as the matrix centers of A[z]
, B[z]
, and (0.001) is a box of edge
length 2 · 0.001. At time t = 0, the input is changed to u ∈ [0.99,1.01] so that the
step response of the output voltage Uout = Ul can be verified. For the modeling of
the transmission line, 20 nodes have been used such that the system has 40 state
variables. The reachable set of Uout is presented in Fig. 9 when computing with matrix zonotopes (dark gray) or interval matrices (light gray). It can be observed that
the matrix zonotope computations are much tighter due to the consideration of the
dependency of the R, L, and C values of each 

In [ ]:
α = 1e-10
Ts = α*T

O = Options(:T = Ts)

A, B = tline(η=20)
s = LinearControlContinuousSystem(A, B)
scale!(s, α)

## Results

## Parameter uncertainties

## References

[AKS11] Althoff, Matthias, Bruce H. Krogh, and Olaf Stursberg. [Analyzing reachability of linear dynamic systems with parametric uncertainties.](http://www.i6.in.tum.de/Main/Publications/Althoff2011b.pdf). Modeling, Design, and Simulation of Systems with Uncertainties. Springer, Berlin, Heidelberg, 2011. 69-94.

[K15] Kluever, Craig A. [Dynamic systems: modeling, simulation, and control.](https://www.wiley.com/en-us/Dynamic+Systems%3A+Modeling%2C+Simulation%2C+and+Control-p-9781118289457). John Wiley & Sons, 2015.

[GLG08] Girard, Antoine, and Colas Le Guernic. [Efficient reachability analysis for linear systems using support functions.]() IFAC Proceedings Volumes 41.2 (2008): 8966-8971.